In [8]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [9]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [10]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  ## Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

In [11]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5?

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

In [96]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30 
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

In [13]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')

In [71]:
##Image collections - growing season

##Landsat 5
ga_2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2000, 2000,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2001 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2001, 2001,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2002 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2002, 2002,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2003 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2003, 2003,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2004 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2004, 2004,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2005 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2005, 2005,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2006 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2006, 2006,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2007 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2007, 2007,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2008 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2008, 2008,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2009 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2009, 2009,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2010 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2010, 2010,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2011 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2011, 2011,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

##Landsat 7
ga_2012 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2012,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

##Landsat 8
ga_2013 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2013, 2013,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2014 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2014, 2014,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2015 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2015, 2015,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2016 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2016, 2016,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2017 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2017, 2017,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2018, 2018,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2019, 2019,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

ga_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2020, 2020,'year')) \
    .filter(ee.Filter.calendarRange(10, 10,'month')) \
    .filterBounds(fc_all)

print(ga_2020.size().getInfo())

6


In [72]:
##Cloud masking and adding (then masking) FLATS - will act as water mask too! Don't need Hansen dataset

##addFLATS - Landsat 5
mean00 = ga_2000.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean01 = ga_2001.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean02 = ga_2002.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean03 = ga_2003.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean04 = ga_2004.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean05 = ga_2005.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean06 = ga_2006.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean07 = ga_2007.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean08 = ga_2008.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean09 = ga_2009.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean10 = ga_2010.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()
mean11 = ga_2011.map(maskL5sr).map(addFLATSL5).map(maskFLATS).mean()

##addFLATS - Landsat 7/8
mean12 = ga_2012.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean13 = ga_2013.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean14 = ga_2014.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean15 = ga_2015.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean16 = ga_2016.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean17 = ga_2017.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean18 = ga_2018.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean19 = ga_2019.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()
mean20 = ga_2020.map(maskL5sr).map(addFLATS).map(maskFLATS).mean()

Map.addLayer(mean00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, '2000')

Map.addLayer(mean20, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, '2020')

Map.addLayer(fc_all, {}, 'All plots')

Map

Map(bottom=27376479.0, center=[31.435460316649934, -81.34075621855108], controls=(WidgetControl(options=['posi…

In [73]:
##Mean - growing season! Also consider: mean - sampling month; mean - 2 months prior; etc. The more included, the better

df00 = geemap.ee_to_pandas(rasterExtraction(mean00))
df01 = geemap.ee_to_pandas(rasterExtraction(mean01))
df02 = geemap.ee_to_pandas(rasterExtraction(mean02))
df03 = geemap.ee_to_pandas(rasterExtraction(mean03))
df04 = geemap.ee_to_pandas(rasterExtraction(mean04))
df05 = geemap.ee_to_pandas(rasterExtraction(mean05))
df06 = geemap.ee_to_pandas(rasterExtraction(mean06))
df07 = geemap.ee_to_pandas(rasterExtraction(mean07))
df08 = geemap.ee_to_pandas(rasterExtraction(mean08))
df09 = geemap.ee_to_pandas(rasterExtraction(mean09))
df10 = geemap.ee_to_pandas(rasterExtraction(mean10))
df11 = geemap.ee_to_pandas(rasterExtraction(mean11))
df12 = geemap.ee_to_pandas(rasterExtraction(mean12))
df13 = geemap.ee_to_pandas(rasterExtraction(mean13))
df14 = geemap.ee_to_pandas(rasterExtraction(mean14))
df15 = geemap.ee_to_pandas(rasterExtraction(mean15))
df16 = geemap.ee_to_pandas(rasterExtraction(mean16))
df17 = geemap.ee_to_pandas(rasterExtraction(mean17))
df18 = geemap.ee_to_pandas(rasterExtraction(mean18))
df19 = geemap.ee_to_pandas(rasterExtraction(mean19))
df20 = geemap.ee_to_pandas(rasterExtraction(mean20)) ##There is no 2020 included, yet

display(df20) ##NOTE: Has all years, only want the specified year

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,B2,B3,B10,B4,B11,B5,B6,B7,flats,B1
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,0.02200,0.02960,0.29415,0.02665,0.29215,0.0828,0.06080,0.03590,0.003652,0.01935
1,1,A1,2001,1,1,31.538627,424.416,18,-81.422539,10/17/2001,0.02200,0.02960,0.29415,0.02665,0.29215,0.0828,0.06080,0.03590,0.003652,0.01935
2,1,A1,2002,1,1,31.538627,259.784,16,-81.422539,10/16/2002,0.02200,0.02960,0.29415,0.02665,0.29215,0.0828,0.06080,0.03590,0.003652,0.01935
3,1,A1,2004,1,1,31.538627,436.332,18,-81.422539,10/25/2004,0.02200,0.02960,0.29415,0.02665,0.29215,0.0828,0.06080,0.03590,0.003652,0.01935
4,1,A1,2005,1,1,31.538627,649.300,25,-81.422539,10/24/2005,0.02200,0.02960,0.29415,0.02665,0.29215,0.0828,0.06080,0.03590,0.003652,0.01935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2527,8,A7,2005,1,4,31.307760,1113.108,67,-81.415721,10/23/2005,0.02570,0.04600,0.29570,0.04930,0.29310,0.1326,0.08800,0.04330,0.000069,0.02070
2528,8,A8,2015,1,3,31.307874,582.536,8,-81.415747,10/23/2015,0.02570,0.04600,0.29570,0.04930,0.29310,0.1326,0.08800,0.04330,0.000069,0.02070
2529,8,A8,2018,1,3,31.307874,636.512,7,-81.415747,10/15/2018,0.02570,0.04600,0.29570,0.04930,0.29310,0.1326,0.08800,0.04330,0.000069,0.02070
2530,8,AX,2012,2,1,31.309053,0.000,1,-81.415765,10/12/2012,0.04555,0.05515,0.29360,0.05990,0.29030,0.1692,0.17575,0.10045,0.000001,0.04375


In [94]:
##SEE IF YOU CAN AUTOMATE THIS SECTION based on year

##DEM extraction and merging
dem_vals = geemap.ee_to_pandas(demExtraction(dem))

# display(dem_vals)

dfx = pd.merge(df00, dem_vals, how = 'left')

##Daymet
daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
     .mean()

# sn_daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
#      .filter(ee.Filter.date('2000-04-01', '2000-10-31')) \
#      .mean()

daymet_vals = geemap.ee_to_pandas(rasterExtraction(daymet))

# display(daymet_vals) ##One site is still missing, so approximately 200 measurements are left! Need to fix

df2000 = pd.merge(dfx, daymet_vals, how = 'right')
df2000 = df2000[df2000['Year'] == 2000]

# display(df2000)

##Current processing notes: Change all details in chunk to desired year. Run, then concat. 9 things to change including display!

# df = pd.concat(
#     [
#         df2000,df2001,df2002,df2004,df2005,df2006,df2007,df2008,df2009,df2010,df2011,df2012,df2013,df2014,df2015,
#         df2016,df2017,df2018,df2019,df2020
#     ]
# )

# display(df) ##Voila

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,flats,B1,elevation,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,0.011329,0.036033,0.465147,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,NaN,NaN,NaN,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938
31,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,0.000368,0.031500,0.431135,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938
45,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,0.047019,0.037450,0.762014,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938
64,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,0.004639,0.040400,0.293990,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,7,A4,2000,2,6,31.334649,705.896,16,-81.478284,10/13/2000,...,0.026145,0.031850,0.904524,0,25.100000,345.478119,14.221671,1780.730225,2.525425,43200.085938
2794,7,A4,2000,1,7,31.334598,893.676,13,-81.478510,10/13/2000,...,0.002905,0.026800,0.574240,0,25.100000,345.478119,14.221671,1780.730225,2.525425,43200.085938
2813,7,A4,2000,2,7,31.334600,1291.824,9,-81.478147,10/13/2000,...,0.002374,0.034133,1.083815,0,25.100000,345.478119,14.221671,1780.730225,2.525425,43200.085938
2832,7,A4,2000,1,8,31.334524,974.992,11,-81.478495,10/13/2000,...,0.002905,0.026800,0.574240,0,25.100000,345.478119,14.221671,1780.730225,2.525425,43200.085938


,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,elevation,swe,tmax,srad,tmin,vp,prcp,dayl,B10,B11
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,0.465147,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938,NaN,NaN
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,NaN,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938,NaN,NaN
31,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,0.431135,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938,NaN,NaN
45,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,0.762014,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938,NaN,NaN
64,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,0.293990,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,NaN,0,26.559807,333.483643,15.817060,1942.531738,3.804478,43219.898438,NaN,NaN
2875,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,NaN,0,26.559807,333.483643,15.817060,1942.531738,3.804478,43219.898438,NaN,NaN
2879,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,NaN,0,26.559807,333.483643,15.817060,1942.531738,3.804478,43219.898438,NaN,NaN
2883,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,NaN,0,26.559807,333.483643,15.817060,1942.531738,3.804478,43219.898438,NaN,NaN


In [95]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8')

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 5', (df['B4']-df['B3'])/(df['B4']+df['B3']), \
                      (df['B5']-df['B4'])/(df['B5']+df['B4'])) ##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B1'], df['B2'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B2'], df['B3'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B3'], df['B4'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B4'], df['B5'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B5'], df['B6'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 5', df['B7'], df['B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_oct.csv')
# df.to_csv(out_csv, index = False)

In [94]:
##From stackexchange: calculating monthly averages across many years:
#https://gis.stackexchange.com/questions/290892/google-earth-enginesst-by-month-per-year

sst = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI') \
    .select('sst') \
    .filterDate(ee.Date('2013-01-01'), ee.Date('2017-12-31'))

months = ee.List.sequence(1, 12)
years = ee.List.sequence(2013, 2017)

byMonthYear = ee.ImageCollection.fromImages(

def func_saj(y):
    return months.map(function (m) {
      return sst \
        .filter(ee.Filter.calendarRange(y, y, 'year')) \
        .filter(ee.Filter.calendarRange(m, m, 'month')) \
        .mean() \
        .set('month', m).set('year', y)
  })

  years.map(func_saj).flatten())


print(byMonthYear)

SyntaxError: invalid syntax (Temp/ipykernel_11828/3088432858.py, line 13)